In [ ]:
# Install required packages
!pip install faiss-cpu sentence-transformers pymupdf pypdf gradio groq

# ============================
# STEP 1: Imports & Setup
# ============================
import fitz  # PyMuPDF
import faiss
from sentence_transformers import SentenceTransformer
from google.colab import files
import gradio as gr
from groq import Groq

In [ ]:

# 🔑 Set your Groq API Key (get from https://console.groq.com/)
GROQ_API_KEY = "ADD_YOUR_API"
client = Groq(api_key=GROQ_API_KEY)


In [ ]:
# ============================
# STEP 2: Upload PDFs
# ============================
uploaded = files.upload()
pdf_paths = list(uploaded.keys())

texts = []
for pdf_path in pdf_paths:
    try:
        doc = fitz.open(pdf_path)
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text = page.get_text()
            if text.strip():  # only keep non-empty pages
                texts.append({"text": text, "source": pdf_path, "page": page_num + 1})
        doc.close()
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")


Saving CV.pdf to CV (1).pdf


In [ ]:
# ============================
# STEP 3: Create Embeddings + Index
# ============================
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode([t['text'] for t in texts])

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

class Retriever:
    def __init__(self, index, texts, model):
        self.index = index
        self.texts = texts
        self.model = model

    def search(self, query, k=5):
        query_embedding = self.model.encode(query)
        distances, indices = self.index.search(query_embedding.reshape(1, -1), k)
        results = []
        for i in indices[0]:
            results.append(self.texts[i])
        return results

retriever = Retriever(index, texts, model)

In [ ]:
# ============================
# STEP 4: LLaMA-3 Completion Function
# ============================
def llama3_complete(prompt):
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-8b-8192",   # or "llama3-70b-8192" if you want bigger model
        temperature=0.2,
        max_tokens=600
    )
    return chat_completion.choices[0].message.content


In [ ]:
# ============================
# STEP 5: Query Function
# ============================
def ask_paper(question):
    results = retriever.search(question)
    context = "\n".join(
        [f"[{r['source']} Page {r['page']}] {r['text']}" for r in results]
    )

    prompt = f"""
    You are an AI assistant that answers based ONLY on the context below.
    If the answer is not in the context, say: "Not found in the provided papers."

    Question: {question}

    Context:
    {context}
    """

    return llama3_complete(prompt)

In [ ]:

# ============================
# STEP 6: Gradio UI
# ============================
demo = gr.Interface(
    fn=ask_paper,
    inputs="text",
    outputs="text",
    title="📄 Research Paper Q&A (RAG + LLaMA-3)",
    description="Ask questions about your uploaded research papers."
)
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://abf826d2e43c7074a3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
